In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [2]:
ratings_data=pd.read_csv(r"C:\Users\Akhilesh_Abhay\Documents\bigquerykeyfile\archive\ratings.csv")

In [ ]:
ratings_data

In [3]:
movies_data=pd.read_csv(r"C:\Users\Akhilesh_Abhay\Documents\bigquerykeyfile\archive\movies.csv")

In [ ]:
movies_data

In [4]:
merged_data = pd.merge(ratings_data, movies_data, on='movieId')

In [ ]:
merged_data

In [5]:
merged_data = merged_data.drop(['timestamp', 'genres'], axis=1)

In [6]:
merged_data = merged_data.iloc[:10000]

In [7]:
merged_data

,userId,movieId,rating,title
0,1,296,5.0,Pulp Fiction (1994)
1,1,306,3.5,Three Colors: Red (Trois couleurs: Rouge) (1994)
2,1,307,5.0,Three Colors: Blue (Trois couleurs: Bleu) (1993)
3,1,665,5.0,Underground (1995)
4,1,899,3.5,Singin' in the Rain (1952)
...,...,...,...,...
9995,75,736,4.0,Twister (1996)
9996,75,778,3.0,Trainspotting (1996)
9997,75,783,3.0,"Hunchback of Notre Dame, The (1996)"
9998,75,805,3.5,"Time to Kill, A (1996)"


In [8]:
user_item_matrix = merged_data.pivot_table(index='userId', columns='movieId', values='rating')

In [9]:
user_item_matrix

movieId,1,2,3,5,6,7,9,10,11,14,...,182715,182823,187541,187593,189333,195159,200818,200838,203375,203519
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,3.0,3.5,4.5,NaN,NaN,NaN,NaN,NaN,NaN
4,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.5,NaN,3.5,3.5,4.0,5.0,5.0,3.0,4.5,2.5
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
72,NaN,2.0,NaN,NaN,4.0,4.0,NaN,NaN,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73,3.0,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
train_data, test_data = train_test_split(user_item_matrix, test_size=0.2, random_state=42)

In [11]:
# Calculate item-item similarity using cosine similarity
item_similarity = cosine_similarity(train_data.fillna(0))
np.fill_diagonal(item_similarity, 0)  # Set diagonal elements to 0 to avoid self-similarity

In [12]:
item_similarity

array([[0.        , 0.23560906, 0.04800771, ..., 0.43086704, 0.23541711,
        0.06893409],
       [0.23560906, 0.        , 0.09177239, ..., 0.19841798, 0.12828523,
        0.05611591],
       [0.04800771, 0.09177239, 0.        , ..., 0.09496708, 0.06088722,
        0.        ],
       ...,
       [0.43086704, 0.19841798, 0.09496708, ..., 0.        , 0.24782041,
        0.07055914],
       [0.23541711, 0.12828523, 0.06088722, ..., 0.24782041, 0.        ,
        0.08881519],
       [0.06893409, 0.05611591, 0.        , ..., 0.07055914, 0.08881519,
        0.        ]])

In [13]:
def get_recommendations(user_id, top_n=5):
    # Get the user's ratings and item similarity
    user_ratings = train_data.loc[user_id].values.reshape(1, -1)
    similarity_scores = item_similarity.dot(user_ratings.T).reshape(-1)

    # Sort the movies based on similarity scores
    sorted_indices = np.argsort(similarity_scores)[::-1]
    top_indices = sorted_indices[:top_n]

    # Get the recommended movie IDs
    recommended_movie_ids = user_item_matrix.columns[top_indices]

    # Get the movie titles
    recommended_movies = movies_data[movies_data['movieId'].isin(recommended_movie_ids)]

    return recommended_movies

In [16]:
user_id=23
user_ratings = train_data.loc[user_id].values.reshape(1, -1)
similarity_scores = item_similarity.dot(user_ratings.T).reshape(-1)

ValueError: shapes (60,60) and (3287,1) not aligned: 60 (dim 1) != 3287 (dim 0)

In [17]:
user_ratings

array([[nan, nan,  5., ..., nan, nan, nan]])